### Basic Setup
Here we check the sage version, configure some settings, define a 2-manifold $M$ with cylidrical coordinate chart $Y$, and a riemannian metric. The naming convention python variables (not variables used in the actual expressions) is `CamelCase` for objects defined on the manifold, and `snake_case` (with a trailing underscore for functions) for generic symbolic expressions. For rank $(0, 2)$ tensors, we default to the purely covariant form (as that is what we have regularization conditions for), denote the $(1, 1)$ version as `X_mat` and denote the rank $(2, 0)$ version as `X_con`.

In [1]:
# Check version
version()

'SageMath version 9.5, Release Date: 2022-01-30'

In [2]:
# Reset
reset()
# Setup pretty printing
%display latex

In [3]:
# Define a differentiable manifold of dimension 2 over real numbers
M = Manifold(2, 'M', latex_name=r'\mathcal{M}', start_index=1)
# Define a cylindrical chart on the manifold
Chart.<rho, z> = M.chart(r'rho:\rho z:z')
# Cache frame
Chartf = Chart.frame()

In [4]:
# Declare a metric on the manifold
G = M.riemannian_metric('g')

# Components of Metric
g_rr_ = function('grr', latex_name='g_{rr}')(rho, z)
g_zz_ = function('gzz', latex_name='g_{zz}')(rho, z)
g_rz_ = function('grz', latex_name='g_{rz}')(rho, z)

# Metric is symmetric
G[1, 1], G[2, 2] = g_rr_, g_zz_
G[1, 2] = g_rz_

# Invert metric
G_con = G.inverse()

# Print metric
G.display()

g = grr(rho, z) drho⊗drho + grz(rho, z) drho⊗dz + grz(rho, z) dz⊗drho + gzz(rho, z) dz⊗dz

In [5]:
# Compute Ricci Tensor
R = G.ricci()
# And Ricci Scalar
R_trace = G_con['^ij'] * R['_ij']
# And connection coefficients, used for covariant derivative
Nabla = G.connection(name='nabla', latex_name=r'\nabla')
# Make sure metric is compatible
assert(Nabla(G) == 0)

### Variables
Next we define the relavent tensor fields, as well as their components in our default frame.

In [6]:
# Regularization of Metric

s_ = function('s')(rho, z)

Lam = M.scalar_field(name='lam', latex_name=r'\lambda')
Lam.add_expr(rho * exp(rho * s_) * sqrt(g_rr_), chart=Chart)

In [7]:
# Gauge Fields
lapse_ = function('lapse', latex_name=r'\alpha')(rho, z)

shift_r_ = function('shift_r', latex_name=r'\beta^r')(rho, z)
shift_z_ = function('shift_z', latex_name=r'\beta^z')(rho, z)

Shift = M.tensor_field(1, 0, name='Shift', latex_name=r'\beta')
Shift.add_comp(Chartf)[1] = shift_r_
Shift.add_comp(Chartf)[2] = shift_z_

Lapse = M.scalar_field(name='Lapse', latex_name=r'\alpha')
Lapse.add_expr(lapse_, chart=Chart)

Lapse.display()

Lapse: M → ℝ
   (rho, z) ↦ lapse(rho, z)

In [8]:
# Extrinsic Curvature

# Components
k_rr_ = function('Krr', latex_name='K_{rr}')(rho, z)
k_zz_ = function('Kzz', latex_name='K_{zz}')(rho, z)
k_rz_ = function('Krz', latex_name='K_{rz}')(rho, z)

# Regularity for L
y_ = function('Y')(rho, z)

# l = function('L', latex_name=r'L')(rho, z)

K = M.tensor_field(0, 2, sym=(0, 1), name='K')
K.add_comp(Chartf)[1, 1] = k_rr_
K.add_comp(Chartf)[2, 2] = k_zz_
K.add_comp(Chartf)[1, 2] = k_rz_

K_mat = G_con['^{ij}'] * K['_jk']
K_con = K_mat['^i_k'] * G_con['^{kj}']
K_trace = G_con['^ij'] * K['_ij']

# L = K_\phi^\phi
L = M.scalar_field(name='L', latex_name=r'L')
L.add_expr(k_rr_/g_rr_ + rho * y_, chart=Chart)

In [9]:
# Z4
theta_ = function('theta', latex_name=r'\theta')(rho, z)
Theta = M.scalar_field(name='theta', latex_name=r'\theta')
Theta.add_expr(theta_, chart=Chart)

z_r_ = function('Zr', latex_name=r'Z_r')(rho, z)
z_z_ = function('Zz', latex_name=r'Z_z')(rho, z)

Z = M.tensor_field(0, 1, name='Z')
Z.add_comp(Chartf)[1] = z_r_
Z.add_comp(Chartf)[2] = z_z_

# Vector form of Z
Zv = G_con['^ij'] * Z['_j']

### Constraint equations
Constraint equations that should be satisfied at all times.

In [10]:
term1 = (K_trace^2 - K['_ij'] * K_con['^ij'] + R_trace) / 2
term2 = - Nabla(Nabla(Lam))['_{ij}'] * G_con['^{ij}'] / Lam + K_trace * L

# Hamiltonian
CH = term1 + term2

In [11]:
term1 = Nabla(K)['_ijk'] * G_con['^jk'] - Nabla(K_trace + L)
term2 = Nabla(Lam)['_j'] * K_mat['^j_i'] / Lam - Nabla(Lam) * L / Lam

# Momentum Constraint
CM = term1 + term2

### Evolution Equations
Calculations for $\mathcal{L}_n X$ for various dynamical quantities.

In [12]:
term1 = R - Nabla(Nabla(Lam)) / Lam - Nabla(Nabla(Lapse)) / Lapse
term2 = (K_trace + L) * K - 2 * K['_ij'] * K_mat['^j_k']
term3 = 2 * Nabla(Z)['_(ij)'] - 2 * K * Theta

# Extrinsic Curvature
LieK = term1 + term2 + term3

In [13]:
term1 = - Nabla(Nabla(Lam))['_{ij}'] * G_con['^{ij}'] / Lam
term2 = - (Nabla(Lam) * Nabla(Lapse))['_{ij}'] * G_con['^{ij}'] / (Lam * Lapse)
term3 = L * (K_trace + L)
term4 = 2 * Nabla(Lam)['_i'] * Zv['^i'] / Lam  - 2 * L * Theta

# Angular Extrinsic Curvature
LieL = term1 + term2 + term3 + term4

In [14]:
# Metric and Lambda
LieG = -2*K
LieLam = -Lam * L

In [15]:
term1 = CH + (Nabla(Lam) / Lam - Nabla(Lapse) / Lapse)['_i'] * Zv['^i']
term2 = Nabla(Zv)['^i_i'] - (K_trace + L) * Theta

# Theta
LieTheta = term1 + term2

In [16]:
term1 = CM - 2 * K['_ij'] * Zv['^j']
term2 = - Nabla(Lapse) * Theta / Lapse + Nabla(Theta)

# Z
LieZ = term1 + term2

In [17]:
# Actual temporal derivatives including shift terms
G_t = Lapse * LieG + G.lie_derivative(Shift)
K_t = Lapse * LieK + K.lie_derivative(Shift)

Theta_t = Lapse * LieTheta + Theta.lie_derivative(Shift)
Z_t = Lapse * LieZ + Z.lie_derivative(Shift)

### Preprocessing
To avoid transcription errors when transferring between notebooks and code we automatically transform symbolic expressions into `C` code.

In [18]:
import sympy as sp

# Symbolic Variables to replace current functions
var('grr_r grr_z grr_rr grr_zz grr_rz grr')
var('gzz_r gzz_z gzz_rr gzz_zz gzz_rz gzz')
var('grz_r grz_z grz_rr grz_zz grz_rz grz')

var('Krr_r Krr_z Krr')
var('Kzz_r Kzz_z Kzz')
var('Krz_r Krz_z Krz')
var('Y_r Y_z Y')

var('s_r s_z s_rr s_zz s_rz s')
var('lapse_r lapse_z lapse_rr lapse_zz lapse_rz lapse')
var('shiftr_r shiftr_z shiftz_r shiftz_z shiftr shiftz')

var('theta_r theta_z theta')
var('Zr_r Zr_z Zz_r Zz_z Zr Zz')

"""
Preprocesses an expression, replacing all derivatives of a function
and invokations of that functions with an appropriately named variable.
"""
def process_expr(expr):
    expr = expr.simplify_full()
    derives = expr.subs({
        diff(g_rr_, rho): grr_r,
        diff(g_rr_, z): grr_z,
        diff(g_rr_, rho, rho): grr_rr,
        diff(g_rr_, z, z): grr_zz,
        diff(g_rr_, rho, z): grr_rz,
        diff(g_rr_, z, rho): grr_rz,
        
        diff(g_zz_, rho): gzz_r,
        diff(g_zz_, z): gzz_z,
        diff(g_zz_, rho, rho): gzz_rr,
        diff(g_zz_, z, z): gzz_zz,
        diff(g_zz_, rho, z): gzz_rz,
        diff(g_zz_, z, rho): gzz_rz,
        
        diff(g_rz_, rho): grz_r,
        diff(g_rz_, z): grz_z,
        diff(g_rz_, rho, rho): grz_rr,
        diff(g_rz_, z, z): grz_zz,
        diff(g_rz_, rho, z): grz_rz,
        diff(g_rz_, z, rho): grz_rz,
        
        diff(k_rr_, rho): Krr_r,
        diff(k_rr_, z): Krr_z,
        
        diff(k_zz_, rho): Kzz_r,
        diff(k_zz_, z): Kzz_z,
        
        diff(k_rz_, rho): Krz_r,
        diff(k_rz_, z): Krz_z,
        
        diff(s_, rho): s_r,
        diff(s_, z): s_z,
        diff(s_, rho, rho): s_rr,
        diff(s_, z, z): s_zz,
        diff(s_, rho, z): s_rz,
        diff(s_, z, rho): s_rz,
        
        diff(lapse_, rho): lapse_r,
        diff(lapse_, z): lapse_z,
        diff(lapse_, rho, rho): lapse_rr,
        diff(lapse_, z, z): lapse_zz,
        diff(lapse_, rho, z): lapse_rz,
        diff(lapse_, z, rho): lapse_rz,
        
        diff(shift_r_, rho): shiftr_r,
        diff(shift_r_, z): shiftr_z,
        diff(shift_z_, rho): shiftz_r,
        diff(shift_z_, z): shiftz_z,
        
        diff(y_, rho): Y_r,
        diff(y_, z): Y_z,
        
        diff(theta_, rho): theta_r,
        diff(theta_, z): theta_z,
        
        diff(z_r_, rho): Zr_r,
        diff(z_r_, z): Zr_z,
        diff(z_z_, rho): Zz_r,
        diff(z_z_, z): Zz_z,
    })
    
    values = derives.subs({
        g_rr_: grr,
        g_zz_: gzz,
        g_rz_: grz,
        
        k_rr_: Krr,
        k_zz_: Kzz,
        k_rz_: Krz,
        
        lapse_: lapse,
        shift_r_: shiftr,
        shift_z_: shiftz,
        
        s_: s,
        y_: Y,
        theta_: theta,
        z_r_: Zr,
        z_z_: Zz,
    })
    
    return values

In [19]:
def generate_ccode(eqs):
    names = []
    exprs = []
    
    for (name, eq) in eqs:
        names.append(name)
        exprs.append(process_expr(eq)._sympy_().simplify())
        
    subs, final = sp.cse(exprs)
    
    result = ("/*************************************\n" + 
              "This code was generated automatically\n" +
              "using Sagemath and SymPy\n" +
              "**************************************/\n")
    result += "\n// Subexpressions\n"
    
    for (name, expr) in subs:
        code = sp.ccode(expr)
        result += "double " + str(name) + " = " + str(code) + ";\n"
        
    result += "\n// Final Equations\n"
        
    for (name, expr) in zip(names, final):
        result += "double " + str(name) + " = " + str(expr) + ";\n\n"
        
    return result

hyperbolic = [
    ("grr_t", G_t[1, 1].expr()),
    ("gzz_t", G_t[2, 2].expr()),
    ("grz_t", G_t[1, 2].expr()),
    ("Krr_t", K_t[1, 1].expr()),
    ("Kzz_t", K_t[2, 2].expr()),
    ("Krz_t", K_t[1, 2].expr()),
    ("theta_t", Theta_t.expr()),
    ("Zr_t", Z_t[1].expr()),
    ("Zz_t", Z_t[2].expr()),
]

hyperbolic_h = open("hyperbolic.h", 'w')
hyperbolic_h.write(generate_ccode(hyperbolic))
hyperbolic_h.close()